In [2]:
import os
import torch
import mmcv
import mmseg 
from mmengine.runner import Runner
import glob
import sys
import numpy

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"MMSegmentation 버전: {mmseg.__version__}")

config_file = './configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py'

work_dir = './work_dirs/segformer_mission3'

PyTorch 버전: 2.7.1+cu128
CUDA 사용 가능 여부: True
MMSegmentation 버전: 1.2.2


In [3]:
!python3 preprocess_m.py

'../data/Mission3/Training/labels' 폴더의 파일을 변환합니다...
100%|██████████████████████████████████████| 8000/8000 [01:02<00:00, 127.88it/s]
'../data/Mission3/Validation/labels' 폴더의 파일을 변환합니다...
100%|██████████████████████████████████████| 1000/1000 [00:07<00:00, 128.39it/s]


In [4]:
from mmengine.config import Config

cfg = Config.fromfile(config_file)

# env_cfg 부분을 재현성에 맞게 수정
cfg.env_cfg = dict(
    cudnn_benchmark=False,  # 재현성을 위해 False로 변경
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'),
    seed=500,
    deterministic=True)     # 결정적 동작을 위한 옵션 추가

# 수정된 내용을 다시 파일에 저장
cfg.dump(config_file)

print(f"'{config_file}'의 시드 설정이 완료되었습니다.")
print("수정된 내용:")
print(cfg.env_cfg)

'./configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py'의 시드 설정이 완료되었습니다.
수정된 내용:
{'cudnn_benchmark': False, 'mp_cfg': {'mp_start_method': 'fork', 'opencv_num_threads': 0}, 'dist_cfg': {'backend': 'nccl'}, 'seed': 500, 'deterministic': True}


In [23]:
!python3 tools/train.py {config_file} --work-dir {work_dir}

print("\n--- 학습이 완료되었습니다. ---")
print(f"학습 결과 및 체크포인트는 '{work_dir}' 폴더를 확인하세요.")

10/12 22:37:02 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 539689079
    GPU 0: NVIDIA GeForce RTX 5080
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    GCC: gcc (Ubuntu 12.3.0-1ubuntu1~22.04.2) 12.3.0
    PyTorch: 2.8.0+cu128
    PyTorch compiling details: PyTorch built with:
  - GCC 13.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2024.2-Product Build 20240605 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.8
  - NVCC architecture flags: -gencode;arch=compute_70,code=s

In [5]:
sys.modules['numpy._core'] = numpy
sys.modules['numpy._core.multiarray'] = numpy.core.multiarray

print(f"--- '{work_dir}' 폴더에서 Best 체크포인트를 찾습니다... ---")

best_checkpoint_list = glob.glob(os.path.join(work_dir, 'best_*.pth'))

if not best_checkpoint_list:
    print(f"!!! 에러: '{work_dir}' 폴더에서 Best 체크포인트 파일을 찾을 수 없습니다.")
else:
    best_checkpoint_path = best_checkpoint_list[0]
    print(f"성공적으로 파일을 찾았습니다: {best_checkpoint_path}")

    src = best_checkpoint_path
    dst = os.path.splitext(src)[0] + ".weights_only.pth"

    print(f"\n--- 가중치만 추출하여 변환을 시작합니다... ---")
    
    ckpt = torch.load(src, map_location="cpu", weights_only=False)
    state = ckpt.get("state_dict", ckpt)
    torch.save(state, dst)

    weights_only_path = dst
    print(f"\n 변환 완료! 가중치 파일이 저장되었습니다: {weights_only_path}")

--- './work_dirs/segformer_mission3' 폴더에서 Best 체크포인트를 찾습니다... ---
성공적으로 파일을 찾았습니다: ./work_dirs/segformer_mission3/best_mIoU_iter_108000.weights_only.pth

--- 가중치만 추출하여 변환을 시작합니다... ---

 변환 완료! 가중치 파일이 저장되었습니다: ./work_dirs/segformer_mission3/best_mIoU_iter_108000.weights_only.weights_only.pth


In [6]:
checkpoint_paths = glob.glob(os.path.join(work_dir, 'best_*only.pth'))

if not checkpoint_paths:
    raise FileNotFoundError(f"{work_dir} 에서 best checkpoint 파일을 찾을 수 없습니다.")

best_checkpoint = max(checkpoint_paths, key=os.path.getmtime)

print(f"선택된 best checkpoint 파일: {best_checkpoint}")

!python tools/test.py {config_file} {best_checkpoint} --cfg-options resume=False resume_from=None auto_resume=False

print("\n--- 평가가 완료되었습니다. ---")

선택된 best checkpoint 파일: ./work_dirs/segformer_mission3/best_mIoU_iter_108000.weights_only.pth
10/14 20:49:25 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1317409557
    GPU 0: NVIDIA GeForce RTX 5080
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    GCC: gcc (Ubuntu 12.3.0-1ubuntu1~22.04.2) 12.3.0
    PyTorch: 2.7.1+cu128
    PyTorch compiling details: PyTorch built with:
  - GCC 11.2
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2024.2-Product Build 20240605 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability 